## REGISTER  DATA dispo Alimentaie

In [1]:
import pandas as pd
import numpy as np
import pymysql as pymysql
import mysql.connector
from mysql.connector import Error



# Import des données des bilans alimentaires 
veg = pd.read_csv("../data/fr_vegetaux.csv")
ani = pd.read_csv("../data/fr_animaux.csv")


# Ajout de la variable origin
ani["origine"] = "animale"
veg["origine"] = "vegetale"


# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)


# Suppression de ani et veg
del ani, veg


# Transformation de temp en table pivot
data = temp.pivot_table(index=["Zone","Produit","Code Produit","origine"],
                        columns = ["Élément"], values=["Valeur"], aggfunc=sum)

data.columns = ['Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire (Kcal/personne/jour)', 
                'Disponibilité alimentaire en quantité (kg/personne/an)', 
                'Disponibilité de matière grasse en quantité (g/personne/jour)', 
                'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 
                'Exportations - Quantité','Nourriture', 'Pertes', 'Production', 
                'Semences', 'Traitement', 'Variation de stock','Code année']


dispo_alimentaire = data.reset_index()

dispo_alimentaire['Code année'] = "2013"

dispo_alimentaire.head()

,Zone,Produit,Code Produit,origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,Code année
0,Afghanistan,Abats Comestible,2736,animale,NaN,NaN,5.0,1.72,0.20,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,2013
1,Afghanistan,"Agrumes, Autres",2614,vegetale,NaN,NaN,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,2013
2,Afghanistan,Aliments pour enfants,2680,vegetale,NaN,NaN,1.0,0.06,0.01,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,2013
3,Afghanistan,Ananas,2618,vegetale,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2013
4,Afghanistan,Bananes,2615,vegetale,NaN,NaN,4.0,2.70,0.02,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,2013


In [2]:
traitementData =  dispo_alimentaire[['Zone','Produit','Code Produit','origine','Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité alimentaire en quantité (kg/personne/an)','Disponibilité de matière grasse en quantité (g/personne/jour)','Disponibilité de protéines en quantité (g/personne/jour)']]

traitementData = traitementData[traitementData['Disponibilité alimentaire en quantité (kg/personne/an)'] > 0]

traitementData['Disponibilité alimentaire en quantité (Tonnes/personne/an)'] =  traitementData['Disponibilité alimentaire en quantité (kg/personne/an)'] / 1000
                                                                                               

traitementData=traitementData.rename(columns={"Zone":"pays","Code zone":"code_pays"
                                              ,"Disponibilité alimentaire (Kcal/personne/jour)":"dispo_alim_kcal_p_j"
                                              ,"origine":"origin"
                                              ,"Disponibilité alimentaire en quantité (Tonnes/personne/an)":"dispo_alim_tonnes"
                                              ,"Produit":"produit","Code Produit":"code_produit"
                                              ,"Disponibilité de protéines en quantité (g/personne/jour)":"dispo_prot",
                                             "Disponibilité de matière grasse en quantité (g/personne/jour)":"dispo_mat_gr"})    
        
                                                                                               



beforeMergeData = traitementData[['pays','produit','code_produit','origin','dispo_alim_tonnes','dispo_alim_kcal_p_j','dispo_prot','dispo_mat_gr']]

beforeMergeData

,pays,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,Afghanistan,Abats Comestible,2736,animale,0.00172,5.0,0.77,0.20
1,Afghanistan,"Agrumes, Autres",2614,vegetale,0.00129,1.0,0.02,0.01
2,Afghanistan,Aliments pour enfants,2680,vegetale,0.00006,1.0,0.03,0.01
4,Afghanistan,Bananes,2615,vegetale,0.00270,4.0,0.05,0.02
5,Afghanistan,"Beurre, Ghee",2740,animale,0.00117,23.0,0.03,2.61
...,...,...,...,...,...,...,...,...
15697,Îles Salomon,Viande de Suides,2733,animale,0.00470,45.0,1.41,4.28
15698,Îles Salomon,Viande de Volailles,2734,animale,0.00334,11.0,1.14,0.69
15699,Îles Salomon,"Viande, Autre",2735,animale,0.00006,0.0,0.04,NaN
15700,Îles Salomon,Vin,2655,vegetale,0.00007,0.0,NaN,NaN


In [3]:
df_population = pd.read_csv("../data/fr_population.csv")

population = df_population[['Zone','Valeur','Code zone','Année']]
population = population[population['Valeur'] > 0]
population['Valeur']=population['Valeur']*1000
population=population.rename(columns={"Zone":"pays","Code zone":"code_pays","Année":"annee","Valeur":"population"})
population.drop_duplicates(subset=['pays'], keep= False)
population=population.loc[population['pays']!= 'Chine',:]

population =  population[['pays','code_pays','annee','population']]

population



,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000
...,...,...,...,...
170,Venezuela (République bolivarienne du),236,2013,30405000
171,Viet Nam,237,2013,91680000
172,Yémen,249,2013,24407000
173,Zambie,251,2013,14539000


In [4]:
mergeDataframe = beforeMergeData.merge(population,on="pays")


mergeDataframe

,pays,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr,code_pays,annee,population
0,Afghanistan,Abats Comestible,2736,animale,0.00172,5.0,0.77,0.20,2,2013,30552000
1,Afghanistan,"Agrumes, Autres",2614,vegetale,0.00129,1.0,0.02,0.01,2,2013,30552000
2,Afghanistan,Aliments pour enfants,2680,vegetale,0.00006,1.0,0.03,0.01,2,2013,30552000
3,Afghanistan,Bananes,2615,vegetale,0.00270,4.0,0.05,0.02,2,2013,30552000
4,Afghanistan,"Beurre, Ghee",2740,animale,0.00117,23.0,0.03,2.61,2,2013,30552000
...,...,...,...,...,...,...,...,...,...,...,...
12005,Îles Salomon,Viande de Suides,2733,animale,0.00470,45.0,1.41,4.28,25,2013,561000
12006,Îles Salomon,Viande de Volailles,2734,animale,0.00334,11.0,1.14,0.69,25,2013,561000
12007,Îles Salomon,"Viande, Autre",2735,animale,0.00006,0.0,0.04,NaN,25,2013,561000
12008,Îles Salomon,Vin,2655,vegetale,0.00007,0.0,NaN,NaN,25,2013,561000


In [5]:
calculationRation =  mergeDataframe

calculationRation['Dispokcal'] = (calculationRation['dispo_alim_kcal_p_j'] * mergeDataframe['population']) * 365

calculationRation

,pays,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr,code_pays,annee,population,Dispokcal
0,Afghanistan,Abats Comestible,2736,animale,0.00172,5.0,0.77,0.20,2,2013,30552000,5.575740e+10
1,Afghanistan,"Agrumes, Autres",2614,vegetale,0.00129,1.0,0.02,0.01,2,2013,30552000,1.115148e+10
2,Afghanistan,Aliments pour enfants,2680,vegetale,0.00006,1.0,0.03,0.01,2,2013,30552000,1.115148e+10
3,Afghanistan,Bananes,2615,vegetale,0.00270,4.0,0.05,0.02,2,2013,30552000,4.460592e+10
4,Afghanistan,"Beurre, Ghee",2740,animale,0.00117,23.0,0.03,2.61,2,2013,30552000,2.564840e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
12005,Îles Salomon,Viande de Suides,2733,animale,0.00470,45.0,1.41,4.28,25,2013,561000,9.214425e+09
12006,Îles Salomon,Viande de Volailles,2734,animale,0.00334,11.0,1.14,0.69,25,2013,561000,2.252415e+09
12007,Îles Salomon,"Viande, Autre",2735,animale,0.00006,0.0,0.04,NaN,25,2013,561000,0.000000e+00
12008,Îles Salomon,Vin,2655,vegetale,0.00007,0.0,NaN,NaN,25,2013,561000,0.000000e+00


In [6]:
mergeDataframe =  mergeDataframe[['pays','code_pays','annee','produit','code_produit','origin','dispo_alim_tonnes','dispo_alim_kcal_p_j','dispo_prot','dispo_mat_gr']]



mergeDataframe['dispo_mat_gr'].fillna(0, inplace=True)
mergeDataframe['dispo_alim_tonnes'].fillna(0, inplace=True)
mergeDataframe['dispo_prot'].fillna(0, inplace=True)
mergeDataframe['dispo_alim_kcal_p_j'].fillna(0, inplace=True)



mergeDataframe

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,pays,code_pays,annee,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,Afghanistan,2,2013,Abats Comestible,2736,animale,0.00172,5.0,0.77,0.20
1,Afghanistan,2,2013,"Agrumes, Autres",2614,vegetale,0.00129,1.0,0.02,0.01
2,Afghanistan,2,2013,Aliments pour enfants,2680,vegetale,0.00006,1.0,0.03,0.01
3,Afghanistan,2,2013,Bananes,2615,vegetale,0.00270,4.0,0.05,0.02
4,Afghanistan,2,2013,"Beurre, Ghee",2740,animale,0.00117,23.0,0.03,2.61
...,...,...,...,...,...,...,...,...,...,...
12005,Îles Salomon,25,2013,Viande de Suides,2733,animale,0.00470,45.0,1.41,4.28
12006,Îles Salomon,25,2013,Viande de Volailles,2734,animale,0.00334,11.0,1.14,0.69
12007,Îles Salomon,25,2013,"Viande, Autre",2735,animale,0.00006,0.0,0.04,0.00
12008,Îles Salomon,25,2013,Vin,2655,vegetale,0.00007,0.0,0.00,0.00


In [7]:
try:
    connect = mysql.connector.connect(   host='db',
                                         database='health',
                                         user='root',
                                         password='root'
                                  )
    if connect.is_connected():
        cursor = connect.cursor()
        cols = "`,`".join([str(i) for i in mergeDataframe.columns.tolist()])
        
        for i,dispAlim in mergeDataframe.iterrows():

            sql = "INSERT INTO `dispo_alim` (`" +cols + "`) VALUES (" + "%s,"*(len(dispAlim)-1) + "%s)"
            cursor.execute(sql, tuple(dispAlim))
            connect.commit()
            
            
            
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connect.is_connected()):
        cursor.close()
        connect.close()
        print("MySQL connection is closed")

Error while connecting to MySQL 1062 (23000): Duplicate entry '2-2736' for key 'dispo_alim.PRIMARY'
MySQL connection is closed
